In [9]:
import numpy as np
import random

In [10]:
supplies = [1546.2452, 566.3322, 439.4694, 310.1833, 254.8308,
            225.002, 199.7258, 186.8757, 145.0616, 142.1455]

# Demand of each demand city
demands = [1000.0, 922.0, 895.0, 859.0, 838.0, 824.0,
           803.0, 783.0, 778.0, 681.0, 632.0]

supplies_new = [i*(9/4) for i in supplies]
supplies = supplies_new

max_helicopter = [(2.5/10* i) for i in supplies]

max_truck = [7.5/10* i for i in supplies]

In [11]:
MAX_HELICOPTER_CAPACITY = 20
HELICOPTER_SPEED = 200
MAX_TRUCK_CAPACITY = 50
TRUCK_SPEED = 100
HELICOPTER_NUMBER = 10
TRUCK_NUMBER = 500
NUM_DISTRIBUTION_CENTERS = 10
NUM_AFFECTED_CITIES = 11

supplies = [1546.2452, 566.3322, 439.4694, 310.1833, 254.8308,
            225.002, 199.7258, 186.8757, 145.0616, 142.1455]

demands = [1000.0, 922.0, 895.0, 859.0, 838.0, 824.0,
           803.0, 783.0, 778.0, 681.0, 632.0]

distance_matrix = np.array([[492.0, 786.0, 647.0, 898.0, 688.0, 572.0, 652.0, 732.0, 749.0,
                            581.0, 711.0],
                           [547.0, 902.0, 763.0, 1084.0, 743.0, 744.0, 953.0, 884.0, 1051.0,
                            636.0, 798.0],
                           [856.0, 1158.0, 1020.0, 1270.0, 1052.0, 944.0, 1024.0, 1104.0,
                            1121.0, 945.0, 1083.0],
                           [70.0, 424.0, 286.0, 606.0, 265.0, 267.0, 466.0, 407.0, 564.0,
                            159.0, 321.0],
                           [951.0, 1237.0, 1098.0, 1349.0, 1147.0, 1023.0, 1103.0, 1183.0,
                            1200.0, 1040.0, 1162.0],
                           [907.0, 1261.0, 1123.0, 1436.0, 1102.0, 1104.0, 1190.0, 1244.0,
                            1287.0, 995.0, 1158.0],
                           [335.0, 468.0, 330.0, 581.0, 428.0, 255.0, 335.0, 414.0, 432.0,
                            356.0, 393.0],
                           [840.0, 1127.0, 988.0, 1239.0, 1036.0, 913.0, 993.0, 1072.0,
                            1090.0, 929.0, 1051.0],
                           [357.0, 712.0, 573.0, 887.0, 553.0, 555.0, 641.0, 695.0, 738.0,
                            446.0, 609.0],
                           [886.0, 1240.0, 1102.0, 1415.0, 1081.0, 1083.0, 1169.0, 1223.0,
                            1266.0, 974.0, 1137.0]])  

supplies_new = [i*(9/4) for i in supplies]
supplies = supplies_new
max_helicopter = [(2.5/10* i) for i in supplies]
max_truck = [7.5/10* i for i in supplies]

def create_population(size, max_truck, max_helicopter, supplies, demands):
    population = []
    for _ in range(size):
        # Copy maximum capacities
        remaining_truck = max_truck.copy()
        remaining_helicopter = max_helicopter.copy()

        # Shuffle the order of indices for i and j
        indices_i = list(range(10))
        indices_j = list(range(11))

        # Shuffle the order of indices again for i and j
        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for truck transfers with random values
        truck_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, demand, and remaining truck capacity
                max_transfer = min(supplies[i], demands[j], remaining_truck[i])
                truck_transfer = np.random.randint(0, max_transfer + 1)
                truck_transfers[i][j] = truck_transfer
                # Decrease remaining truck capacity
                remaining_truck[i] -= truck_transfer

        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for helicopter transfers based on truck transfers
        helicopter_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, remaining demand, and remaining helicopter capacity
                remaining_supply = supplies[i] - truck_transfers[i][j]
                remaining_demand = demands[j] - truck_transfers[i][j]
                max_transfer = min(remaining_supply, remaining_demand, remaining_helicopter[i])
                helicopter_transfer = np.random.randint(0, max_transfer + 1)  # Generate random helicopter transfer
                helicopter_transfers[i][j] = helicopter_transfer
                # Decrease remaining helicopter capacity
                remaining_helicopter[i] -= helicopter_transfer
        individual = [
            truck_transfers,
            helicopter_transfers
        ]
        population.append(individual)
    return population


def fitness_function(individual):
    truck_transfers = np.array(individual[0])
    helicopter_transfers = np.array(individual[1])
    total_cost = np.sum(distance_matrix * (helicopter_transfers / HELICOPTER_SPEED)) \
                 + np.sum(distance_matrix * (truck_transfers / TRUCK_SPEED))
    print(f"total cost:{total_cost}")
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2:
            return -1e6
        elif sum(truck_transfers[i]) > max_truck[i] or sum(helicopter_transfers[i]) > max_helicopter[i]:
            return -1e6
    return -total_cost


def random_neighbor(individual, max_truck, max_helicopter, supplies, demands):
    truck_transfers, helicopter_transfers = individual

    for i in range(10):
        for j in range(11):
            if random.random() < 0.5:
                truck_transfers[i][j] += np.random.randint(0, 3)
                helicopter_transfers[i][j] += np.random.randint(0, 3)

    # Check if the new solution exceeds capacity limits
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2 \
                or sum(truck_transfers[i]) > max_truck[i] \
                or sum(helicopter_transfers[i]) > max_helicopter[i]:
            return individual  # Return the original solution if the new solution exceeds capacity limits

    return [truck_transfers, helicopter_transfers]


def acceptance_probability(energy, new_energy, temperature):
    if new_energy < energy:
        return 1.0
    return np.exp((energy - new_energy) / temperature)


def simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands, num_iterations, max_temperature):
    current_solution = initial_solution
    best_solution = initial_solution
    current_fitness = fitness_function(current_solution)
    best_fitness = current_fitness

    for iteration in range(num_iterations):
        temperature = max_temperature * (1 - iteration / num_iterations)  # Cooling schedule

        new_solution = random_neighbor(current_solution, max_truck, max_helicopter, supplies, demands)
        new_fitness = fitness_function(new_solution)

        if acceptance_probability(current_fitness, new_fitness, temperature) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness

        if new_fitness > best_fitness:
            best_solution = new_solution
            best_fitness = new_fitness

    return best_solution

population_size = 500
initial_solution = create_population(population_size, max_truck, max_helicopter, supplies, demands)[0]
num_iterations = 200
max_temperature = 1.0

best_individual = simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands,
                                     num_iterations, max_temperature)

print("Best Individual (Truck):\n", best_individual[0], "\n")
print("Best Individual (Helicopter):\n", best_individual[1])
print("Best Fitness:", fitness_function(best_individual))


total cost:64770.219999999994
total cost:65456.57
total cost:66223.43500000001
total cost:66868.10500000001
total cost:67581.245
total cost:68174.36
total cost:68828.465
total cost:69674.155
total cost:70411.055
total cost:71143.94
total cost:71917.97
total cost:72608.19499999999
total cost:73483.18
total cost:74137.145
total cost:74930.7
total cost:75728.68
total cost:76343.69499999999
total cost:77008.755
total cost:77750.04500000001
total cost:78435.865
total cost:79108.405
total cost:79858.985
total cost:80429.33999999998
total cost:81224.37999999999
total cost:81949.735
total cost:82578.53
total cost:83236.995
total cost:83836.25
total cost:84571.30000000002
total cost:85316.265
total cost:86042.545
total cost:86768.2
total cost:87454.0
total cost:88175.09
total cost:88840.49999999999
total cost:89626.915
total cost:90394.87500000001
total cost:91098.28
total cost:91791.015
total cost:92566.60000000002
total cost:93429.26999999999
total cost:94018.25499999998
total cost:94759.1699